# <center><b> Python Programming in Energy Science I</b></center>

## <center> Assignment A2 </center>

### <center>[Group 4]</center>
<center> Juan Manuel Boullosa Novo </center>

-------------------------------

### E1. Data analysis

In [82]:
class WindAnalysis:
    def __init__(self, filepath):
        self.datestamps_list, self.wind_speed_vectors_list = self._read_csv(filepath)
        self.magnitude_list = [self._vector_magnitude(vector) for vector in self.wind_speed_vectors_list]
        self.mean, self.mean_mag, self.std = self._mean_vector()
        self.is_positive, self.is_over_25 = self._check_wind_speeds()
        self.min, self.max = self._find_min_max()
    '''
    PRIVATE methods called by the constructor or other class methods:
    '''
    def _read_csv(self, filepath):
        with open(filepath, 'r') as csv:
            next(csv)  # skip header
            data = [line.strip().split(',') for line in csv]
        # separate dates from wind speed vectors
        datestamps_list = [row[0] for row in data]
        wind_speed_vectors_list = [(float(row[1]), float(row[2])) for row in data]
        return datestamps_list, wind_speed_vectors_list
    
    def _vector_magnitude(self, vector):
        return (vector[0] ** 2 + vector[1] ** 2) ** 0.5
    
    def _mean_vector(self):
        list_length = len(self.wind_speed_vectors_list)
        sum_u100 = sum(row[0] for row in self.wind_speed_vectors_list)
        sum_v100 = sum(row[1] for row in self.wind_speed_vectors_list)
        mean_vector = [sum_u100 / list_length, sum_v100 / list_length]
        mean_vector_magnitude = self._vector_magnitude(mean_vector)
        std_dev = self._standard_deviation(mean_vector)
        return mean_vector, mean_vector_magnitude, std_dev
        
    def _check_wind_speeds(self):
        return all(mag > 0 for mag in self.magnitude_list), any(mag > 25 for mag in self.magnitude_list)

    def _find_min_max(self):
        return min(self.magnitude_list), max(self.magnitude_list)
    '''
    PRIVATE HELPER methods that are not called by the user:
    '''    
    def _standard_deviation(self, mean_vector):
        # Handled by _mean_vector() method
        squared_differences = [(vector[0] - mean_vector[0]) ** 2 \
            + (vector[1] - mean_vector[1]) ** 2 for vector in self.wind_speed_vectors_list]
        variance = sum(squared_differences) / len(squared_differences)
        return variance ** 0.5
    
    def _time_difference(self, date1, date2):
        # Calculate the difference in seconds between two dates
        year1, month1, day1, hour1, minute1, second1 =\
            map(int, date1.replace('-', ' ').replace(':', ' ').split())
        year2, month2, day2, hour2, minute2, second2 =\
            map(int, date2.replace('-', ' ').replace(':', ' ').split())

        time1 = second1 + minute1*60 + hour1*3600 + day1*86400 + month1*2592000 + year1*31104000
        time2 = second2 + minute2*60 + hour2*3600 + day2*86400 + month2*2592000 + year2*31104000
        return abs(time2 - time1)
    
    def _s_to_hms(self, seconds):
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = seconds % 60
        return hours, minutes, seconds
    
    def _float_range(self, start, stop, step):
        # Custom generator function to create a range of floats
        while start < stop:
            yield start
            start += step
    '''
    PUBLIC methods that can be called by the user:
    '''
    def bin_counter(self, bin_size=1):
        # Sorts wind speeds into bins (dict) of size bin_size from min to max
        bins = {}
        for bin in self._float_range(self.min, self.max, bin_size):
            bins[bin] = sum(1 for magnitude in self.magnitude_list if bin <= magnitude < bin+1)
        return bins

    def longest_period_greater_than(self, threshold=10):
        # Threshold is the minimum wind speed to be considered in m/s
        longest_period, current_period = 0, 0
        previous_date = None
        
        for i in range(len(self.wind_speed_vectors_list)):
            # Iterate through each wind speed vector
            this_magnitude = self.magnitude_list[i]
            this_date = self.datestamps_list[i]
            
            if this_magnitude >= threshold:
                if previous_date is not None:
                    current_period += self._time_difference(previous_date, this_date)
                previous_date = this_date
            else:
                # Update longest_period when wind speed drops below threshold and reset current_period
                longest_period = max(longest_period, current_period)
                current_period = 0

        longest_period = self._s_to_hms(max(longest_period, current_period))
        return longest_period
'''
MAIN PROGRAM:
'''
# Call instance of class with csv file path as constructor argument:
wind_analysed = WindAnalysis('winddata.csv')
# Call optional methods:
bins = wind_analysed.bin_counter(1)
longest_period10 = wind_analysed.longest_period_greater_than(10)
longest_period15 = wind_analysed.longest_period_greater_than(15)
longest_period25 = wind_analysed.longest_period_greater_than(25)

# Print results:
print(f'Average wind speed vector: [{wind_analysed.mean[0]:.4f}, {wind_analysed.mean[1]:.4f}] m/s')
print(f'Mean windspeed: {wind_analysed.mean_mag:.4f} m/s')
print(f'Standard deviation: {wind_analysed.std:.4f}')
print(f'All wind speeds are positive: {wind_analysed.is_positive}')
print(f'Any wind speed is over 25 m/s: {wind_analysed.is_over_25}')
print(f'Minimum and maximum wind speeds: [{wind_analysed.min:.4f} m/s, {wind_analysed.max:.4f} m/s]')

# Print wind speed bins with format:
print(f'\nCount of wind speeds in bins of size 1 m/s:\n')
print(f'{"Bin [low, high]":<}{"#":>6} \n {"-"*20}')
for this_bin, value in bins.items():
    print(f'[{format(this_bin, ">5.2f")}, {this_bin+1:>5.2f}] -> {value:>3}')

# Print longest periods of wind speed greater than 10, 15 and 25 m/s:
print(f'\nLongest period of wind speed greater than 10 m/s:\n\
    [{longest_period10[0]} hours, {longest_period10[1]} minutes, {longest_period10[2]} seconds]')
print(f'\nLongest period of wind speed greater than 15 m/s:\n\
    [{longest_period15[0]} hours, {longest_period15[1]} minutes, {longest_period15[2]} seconds]')
print(f'\nLongest period of wind speed greater than 25 m/s:\n\
    [{longest_period25[0]} hours, {longest_period25[1]} minutes, {longest_period25[2]} seconds]')

Average wind speed vector: [1.9637, 0.4262] m/s
Mean windspeed: 2.0095 m/s
Standard deviation: 6.3649
All wind speeds are positive: True
Any wind speed is over 25 m/s: False
Minimum and maximum wind speeds: [0.0621 m/s, 23.1355 m/s]

Count of wind speeds in bins of size 1 m/s:

Bin [low, high]     # 
 --------------------
[ 0.06,  1.06] -> 134
[ 1.06,  2.06] -> 408
[ 2.06,  3.06] -> 628
[ 3.06,  4.06] -> 619
[ 4.06,  5.06] -> 616
[ 5.06,  6.06] -> 564
[ 6.06,  7.06] -> 459
[ 7.06,  8.06] -> 399
[ 8.06,  9.06] -> 425
[ 9.06, 10.06] -> 318
[10.06, 11.06] -> 174
[11.06, 12.06] -> 131
[12.06, 13.06] ->  86
[13.06, 14.06] ->  46
[14.06, 15.06] ->  34
[15.06, 16.06] ->  16
[16.06, 17.06] ->   9
[17.06, 18.06] ->   9
[18.06, 19.06] ->   4
[19.06, 20.06] ->   6
[20.06, 21.06] ->   1
[21.06, 22.06] ->   1
[22.06, 23.06] ->   0
[23.06, 24.06] ->   1

Longest period of wind speed greater than 10 m/s:
    [737 hours, 0 minutes, 0 seconds]

Longest period of wind speed greater than 15 m/s:
    [396

### E2. Database

In [6]:
table = {
    "Julia": {"Age": 3, "Color": "Green"},
    "Jim": {"Age": 32, "Color": "Red"},
    "Marco": {"Age": 16, "Color": "Green"},
    "Denise": {"Age": 23, "Color": "Blue"},
    "Paula": {"Age": 28, "Color": "Red"},
    "Louis": {"Age": 42, "Color": "Yellow"}
}

def get_person_info(table, name):
    if name not in table:
        raise KeyError(f"No person named {name} in the database.")
    return table[name]['Age'], table[name]['Color']

def get_people_by_color(table, color):
    people = [name for name, person in table.items() if person['Color'] == color]
    if not people:
        raise ValueError(f"No person associated with color {color} in the database.")
    return people

def get_people_by_age_range(table, min_age, max_age):
    people = [name for name, person in table.items() if min_age <= person['Age'] <= max_age]
    if not people:
        return (f"No person between {min_age} and {max_age} years old in the database.")
    return people

print("People:",*table)

name = input("Enter name: ")
age, color = get_person_info(table, name)
print(f"\n{name} is {age} years old and likes color {color}")

color = input("Enter color: ")
people = get_people_by_color(table, color)
print(f"\nPeople who like color {color} are:")
print(', '.join(people))

min_age, max_age = map(int, input("Enter min and max age: ").split())
people = get_people_by_age_range(table, min_age, max_age)
print(f"\nPeople between {min_age} and {max_age} years old are:")
print(*people)

People: Julia Jim Marco Denise Paula Louis

Julia is 3 years old and likes color Green

People who like color Red are:
Jim, Paula

People between 20 and 50 years old are:
Jim Denise Paula Louis


### E3. Poetry generator

In [103]:
from numpy.random import randint

subjects = ["Dog", "Cat", "Bird", "Fish", "Elephant", "Lion", "Tiger", "Bear"]
verbs = ["runs", "jumps", "flies", "swims", "eats", "sleeps", "plays", "sings"]
objects = ["ball", "fish", "mouse", "tree", "leaf", "rock", "stick", "flower"]
comments = ["quickly.", "slowly,", "happily!", "sadly?", "quietly.", "loudly,", "carefully!", "recklessly?"]

sentence_data = {
    "subject": subjects,
    "verb": verbs,
    "object": objects,
    "comment": comments
}

def random_word_generator(list):
    yield list[randint(0, len(list))]

def random_sentence(data):
    subject = random_word_generator(data["subject"])
    verb = random_word_generator(data["verb"])
    object = random_word_generator(data["object"])
    comment = random_word_generator(data["comment"])
    return f"{next(subject)} {next(verb)} {next(object)} {next(comment)}"

for i in range(3):
    print(random_sentence(sentence_data))

Lion sings ball happily!
Dog plays rock quickly.
Elephant eats fish recklessly?


### E4. Classic phone book

In [19]:
class PhoneBook:
    def __init__(self):
        self.phone_book = {}
        
    def add_contact(self, name, number):
        if name in self.phone_book:
            raise ValueError(f"{name} already exists in the phone book. Please enter a new name.")
        self.phone_book[name] = number
        
    def list_contacts(self):
        if not self.phone_book:
            raise ValueError("The phone book is empty.")
        return "\n".join(f"{name}" for name in self.phone_book.keys())
            
    def get_number(self, name):
        if name not in self.phone_book:
            raise ValueError(f"{name} does not exist in the phone book. Please enter another name.")
        return self.phone_book[name]

phone_book = PhoneBook()
commands = {'add': phone_book.add_contact, 'list': phone_book.list_contacts, 'get': phone_book.get_number}

while True:
        command = input("Enter command (add, list, get, quit): ")
        
        if command == 'quit':
            break
        
        elif command == 'add':
            name = input("Enter name: ")
            number = input("Enter number: ")
            commands[command](name, number)
            
        elif command == 'list':
            print(commands[command]())
            
        elif command == 'get':
            name = input("Enter name: ")
            number = commands[command](name)
            print(f"{name}: {number}")
            
        else:
            print("Invalid command.")

Invalid command.
Invalid command.
Juan
Pepe
Invalid command.
Pepe: 5


### E5. String analysis

In [191]:
def count_lines(text):
    return len(text.splitlines())

def count_words(text):
    return len(text.split())

def count_chars(text):
    return len(text)

def count_occurrences(text, phrase):
    text_lower = text.lower()
    phrase_lower = phrase.lower()
    return text_lower.count(phrase_lower)

def replace_word(text, old_word, new_word):
    return text.replace(old_word, new_word)

def find_sentences(text):
    sentences = text.split('.')
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

def find_sentences_with_word(text, word):
    sentences = find_sentences(text)
    word_sentences = [sentence for sentence in sentences if word in sentence.split()]
    return word_sentences

wordlist = ["python", "lancelot", "holy grail"]

with open("holy_grail.txt", "r") as csv:
    text_string = csv.read()
    
    print(f"Numer of lines: {count_lines(text_string)}")
    print(f"Numer of words: {count_words(text_string)}")
    print(f"Numer of characters: {count_chars(text_string)}\n")
    for word in wordlist:
        print(f"Numer of times '{word}' appears: {count_occurrences(text_string, word)}")
    
    changed_text = replace_word(text_string, '?', '.')
    changed_text = replace_word(changed_text, '!', '.')
    new_word = "Cobra"
    changed_text = replace_word(changed_text, "Python", new_word)
    
    cobra_sentences = find_sentences_with_word(changed_text, new_word)
    print(f"\n{len(cobra_sentences)} \'Sentences\' with the word \"{new_word}\":\n")
    print(*cobra_sentences, sep='\n---------------------------------------\n')

Numer of lines: 169
Numer of words: 3436
Numer of characters: 20978

Numer of times 'python' appears: 31
Numer of times 'lancelot' appears: 11
Numer of times 'holy grail' appears: 18

13 'Sentences' with the word "Cobra":

Monty Cobra and the Holy Grail
From Wikipedia, the free encyclopedia
Jump to navigationJump to search
Monty Cobra and the Holy Grail
Monty-Cobra-1975-poster
---------------------------------------
png
British theatrical releases banner
Directed by	
Terry Gilliam
Terry Jones
Produced by	
Mark Forstater
Michael White
Written by	Monty Cobra
Starring	
Graham Chapman
John Cleese
Terry Gilliam
Eric Idle
Terry Jones
Michael Palin
Music by	
Dewolfe
Neil Innes
Cinematography	Terry Bedford
Edited by	John Hackney
Production
companies	
Cobra (Monty) Pictures
Michael White Productions
National Film Trustee Company
Distributed by	EMI Films
Release date	
April 3, 1975 (United Kingdom)
Running time	92 minutes[1]
Country	United Kingdom
Budget	$400,000[2]
Box office	$5 million[2]
Mont

### E6. Fibonacci sequence

In [190]:
PI = 3.1415

def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)
    
def fibonacci_sequence(n):
    return [fibonacci(i) for i in range(n+1)]

def fibonacci_sequence_generator(n):
    for i in range(n+1):
        yield fibonacci(i)

def circle_area(radius):
    return PI * radius**2

def circle_areas_fibonacci(n):
    # Calculate area for each Fibonacci number
    return map(circle_area, fibonacci_sequence(n))

n = 10
print(f"The first {n} Fibonacci numbers:\n{fibonacci_sequence(n)}\n")
print(f"Area of circles with radii equal to the first {n} Fibonacci numbers are:\
    \n{list(circle_areas_fibonacci(n))}\n")
print(f"Fibonacci numbers up to {n} using a generator function:\
    \n{list(fibonacci_sequence_generator(n))}")

The first 10 Fibonacci numbers:
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

Area of circles with radii equal to the first 10 Fibonacci numbers are:    
[0.0, 3.1415, 3.1415, 12.566, 28.273500000000002, 78.53750000000001, 201.056, 530.9135, 1385.4015000000002, 3631.574, 9503.0375]

Fibonacci numbers up to 10 using a generator function:    
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
